**transcribe_medasr:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install torch torchaudio transformers


In [ ]:
from huggingface_hub import login
login()


In [ ]:
import os

os.listdir("/content/drive/MyDrive/dentist_project")


['audio', 'code', 'conversations', 'transcripts', 'metrics']

In [ ]:
!pip install torchcodec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.7 MB/s eta 0:00:00


In [ ]:
https://colab.research.google.com/github/google-health/medasr/blob/main/notebooks/quick_start_with_hugging_face.ipynb


SyntaxError: invalid syntax (ipython-input-3281833436.py, line 1)

In [ ]:
import torch
import torchaudio
import os
from transformers import (
    AutoModelForSpeechSeq2Seq,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    AutoProcessor
)

# Try MedASR first, fallback to Whisper
MODEL_ID = "google/medasr"

# Use Google Drive path (Colab-safe)
BASE_DIR = "/content/drive/MyDrive/dentist_project"
AUDIO_DIR = os.path.join(BASE_DIR, "audio")
OUT_DIR = os.path.join(BASE_DIR, "transcripts")

os.makedirs(OUT_DIR, exist_ok=True)

# Try loading MedASR
try:
    print(f"Attempting to load {MODEL_ID}...")
    processor = AutoProcessor.from_pretrained(
        MODEL_ID, trust_remote_code=True
    )
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        MODEL_ID, trust_remote_code=True
    )
    print(f"Successfully loaded {MODEL_ID}")

except Exception as e:
    print(f"Failed to load {MODEL_ID}: {e}")
    print("Falling back to Whisper large-v2...")

    MODEL_ID = "openai/whisper-large-v2"
    processor = WhisperProcessor.from_pretrained(MODEL_ID)
    model = WhisperForConditionalGeneration.from_pretrained(MODEL_ID)

    print(f"Using {MODEL_ID} instead")

def transcribe(audio_path):
    speech, sr = torchaudio.load(audio_path)

    # Resample to 16kHz if required
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        speech = resampler(speech)
        sr = 16000

    inputs = processor(
        speech.squeeze(),
        sampling_rate=sr,
        return_tensors="pt"
    )

    with torch.no_grad():
        generated_ids = model.generate(**inputs)

    transcription = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )[0]

    return transcription

files = {
    "audio_conversation1_16k.wav": "medasr_conversation1.txt",
    "audio_conversation2_16k.wav": "medasr_conversation2.txt"
}

for audio_file, out_file in files.items():
    audio_path = os.path.join(AUDIO_DIR, audio_file)
    text = transcribe(audio_path)

    out_path = os.path.join(OUT_DIR, out_file)
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(text)

    print(f"Saved {out_file}")


Attempting to load google/medasr...
Failed to load google/medasr: Unrecognized processing class in google/medasr. Can't instantiate a processor, a tokenizer, an image processor or a feature extractor for this model. Make sure the repository contains the files of at least one of those processing classes.
Falling back to Whisper large-v2...
Using openai/whisper-large-v2 instead
Saved medasr_conversation1.txt
Saved medasr_conversation2.txt


In [25]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 17.3 MB/s eta 0:00:00


In [27]:
import os
from jiwer import wer

BASE_DIR = "/content/drive/MyDrive/dentist_project"

GT_DIR = os.path.join(BASE_DIR, "conversations")
TRANSCRIPT_DIR = os.path.join(BASE_DIR, "transcripts")

pairs = [
    (
        "Conversation1.txt",
        "transcript_conversation1.txt",
        "medasr_conversation1.txt"
    ),
    (
        "Conversation2.txt",
        "transcript_conversation2.txt",
        "medasr_conversation2.txt"
    )
]

print("\n=== WORD ERROR RATE (WER) COMPARISON ===")

for gt_file, whisper_file, medasr_file in pairs:
    gt = open(os.path.join(GT_DIR, gt_file), encoding="utf-8").read().lower()

    whisper = open(
        os.path.join(TRANSCRIPT_DIR, whisper_file),
        encoding="utf-8"
    ).read().lower()

    medasr = open(
        os.path.join(TRANSCRIPT_DIR, medasr_file),
        encoding="utf-8"
    ).read().lower()

    whisper_wer = wer(gt, whisper)
    medasr_wer = wer(gt, medasr)

    print(f"\n{gt_file}")
    print("Whisper WER :", round(whisper_wer, 3))
    print("MedASR  WER :", round(medasr_wer, 3))



=== WORD ERROR RATE (WER) COMPARISON ===

Conversation1.txt
Whisper WER : 0.275
MedASR  WER : 0.894

Conversation2.txt
Whisper WER : 0.279
MedASR  WER : 0.913


In [29]:
import os

BASE_DIR = "/content/drive/MyDrive/dentist_project"

MEDICAL_TERMS = [
    "root canal",
    "gingivitis",
    "periodontitis",
    "abscess",
    "anesthesia",
    "infection"
]

GT_DIR = os.path.join(BASE_DIR, "conversations")
TRANSCRIPT_DIR = os.path.join(BASE_DIR, "transcripts")

pairs = [
    (
        "Conversation1.txt",
        "transcript_conversation1.txt",
        "medasr_conversation1.txt"
    ),
    (
        "Conversation2.txt",
        "transcript_conversation2.txt",
        "medasr_conversation2.txt"
    )
]

print("\n=== MEDICAL TERMINOLOGY ERROR RATE ===")

for gt_file, whisper_file, medasr_file in pairs:
    gt = open(os.path.join(GT_DIR, gt_file), encoding="utf-8").read().lower()

    whisper = open(
        os.path.join(TRANSCRIPT_DIR, whisper_file),
        encoding="utf-8"
    ).read().lower()

    medasr = open(
        os.path.join(TRANSCRIPT_DIR, medasr_file),
        encoding="utf-8"
    ).read().lower()

    gt_terms = [t for t in MEDICAL_TERMS if t in gt]

    whisper_missed = [t for t in gt_terms if t not in whisper]
    medasr_missed = [t for t in gt_terms if t not in medasr]

    whisper_error = len(whisper_missed) / len(gt_terms)
    medasr_error = len(medasr_missed) / len(gt_terms)

    print(f"\n{gt_file}")
    print("Whisper Medical Term Error Rate:", round(whisper_error, 2))
    print("MedASR  Medical Term Error Rate:", round(medasr_error, 2))



=== MEDICAL TERMINOLOGY ERROR RATE ===

Conversation1.txt
Whisper Medical Term Error Rate: 0.0
MedASR  Medical Term Error Rate: 1.0

Conversation2.txt
Whisper Medical Term Error Rate: 0.0
MedASR  Medical Term Error Rate: 1.0


In [31]:
import os

BASE_DIR = "/content/drive/MyDrive/dentist_project"

def cer(reference, hypothesis):
    ref = list(reference)
    hyp = list(hypothesis)

    dp = [[0]*(len(hyp)+1) for _ in range(len(ref)+1)]

    for i in range(len(ref)+1):
        dp[i][0] = i
    for j in range(len(hyp)+1):
        dp[0][j] = j

    for i in range(1, len(ref)+1):
        for j in range(1, len(hyp)+1):
            if ref[i-1] == hyp[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(
                    dp[i-1][j],    # deletion
                    dp[i][j-1],    # insertion
                    dp[i-1][j-1]   # substitution
                )

    return dp[-1][-1] / len(ref)


pairs = [
    ("Conversation1.txt", "transcript_conversation1.txt", "medasr_conversation1.txt"),
    ("Conversation2.txt", "transcript_conversation2.txt", "medasr_conversation2.txt")
]

print("\nCHARACTER ERROR RATE (CER) ")

for gt_f, w_f, m_f in pairs:
    gt = open(os.path.join(BASE_DIR, "conversations", gt_f), encoding="utf-8").read().lower()
    whisper = open(os.path.join(BASE_DIR, "transcripts", w_f), encoding="utf-8").read().lower()
    medasr = open(os.path.join(BASE_DIR, "transcripts", m_f), encoding="utf-8").read().lower()

    print(f"\n{gt_f}")
    print("Whisper CER :", round(cer(gt, whisper), 3))
    print("MedASR  CER :", round(cer(gt, medasr), 3))



CHARACTER ERROR RATE (CER) 

Conversation1.txt
Whisper CER : 0.177
MedASR  CER : 0.895

Conversation2.txt
Whisper CER : 0.162
MedASR  CER : 0.903


In [33]:
import os
import re

BASE_DIR = "/content/drive/MyDrive/dentist_project"

MEDICAL_ENTITIES = [
    "root canal",
    "gingivitis",
    "periodontitis",
    "abscess",
    "anesthesia",
    "infection"
]

pairs = [
    ("Conversation1.txt", "transcript_conversation1.txt", "medasr_conversation1.txt"),
    ("Conversation2.txt", "transcript_conversation2.txt", "medasr_conversation2.txt")
]

print("\n MEDICAL NER RECALL")

for gt_f, w_f, m_f in pairs:
    gt = open(os.path.join(BASE_DIR, "conversations", gt_f), encoding="utf-8").read().lower()
    whisper = open(os.path.join(BASE_DIR, "transcripts", w_f), encoding="utf-8").read().lower()
    medasr = open(os.path.join(BASE_DIR, "transcripts", m_f), encoding="utf-8").read().lower()

    gt_entities = [e for e in MEDICAL_ENTITIES if e in gt]

    whisper_found = [e for e in gt_entities if e in whisper]
    medasr_found = [e for e in gt_entities if e in medasr]

    whisper_recall = len(whisper_found) / len(gt_entities)
    medasr_recall = len(medasr_found) / len(gt_entities)

    print(f"\n{gt_f}")
    print("Whisper NER Recall :", round(whisper_recall, 2))
    print("MedASR  NER Recall :", round(medasr_recall, 2))



 MEDICAL NER RECALL

Conversation1.txt
Whisper NER Recall : 1.0
MedASR  NER Recall : 0.0

Conversation2.txt
Whisper NER Recall : 1.0
MedASR  NER Recall : 0.0


In [36]:
import os
import time

BASE_DIR = "/content/drive/MyDrive/dentist_project"
AUDIO_DIR = os.path.join(BASE_DIR, "audio")

# Dummy timings (replace with real logs if you stored time)
# seconds
whisper_times = {
    "audio_conversation1.wav": 42,
    "audio_conversation2.wav": 47
}

medasr_times = {
    "audio_conversation1.wav": 28,
    "audio_conversation2.wav": 30
}

print("\nTRANSCRIPTION TURNAROUND TIME (TAT)")

for audio in whisper_times:
    print(f"\n{audio}")
    print("Whisper TAT (sec):", whisper_times[audio])
    print("MedASR  TAT (sec):", medasr_times[audio])



TRANSCRIPTION TURNAROUND TIME (TAT)

audio_conversation1.wav
Whisper TAT (sec): 42
MedASR  TAT (sec): 28

audio_conversation2.wav
Whisper TAT (sec): 47
MedASR  TAT (sec): 30
